In [1]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
import janitor

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
train

,temperature,humidity,light,co2,humidityratio,target
0,22.790000,30.8900,444.0,1372.571429,0.005307,1
1,20.890000,25.4175,0.0,573.000000,0.003879,0
2,22.100000,29.7000,0.0,1092.250000,0.004889,0
3,20.633333,24.2300,0.0,464.333333,0.003639,0
4,20.790000,24.9725,0.0,525.500000,0.003787,0
...,...,...,...,...,...,...
2127,22.660000,30.8300,433.0,1306.500000,0.005254,1
2128,20.790000,24.7900,433.0,668.000000,0.003759,1
2129,21.100000,24.8900,444.0,740.800000,0.003848,1
2130,22.200000,27.5400,538.6,1093.200000,0.004559,1


In [4]:
#train = train.drop(["cabin", "name", "passengerid"], axis=1)
#test = test.drop(["cabin", "name", "passengerid"], axis = 1)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   temperature    2132 non-null   float64
 1   humidity       2132 non-null   float64
 2   light          2132 non-null   float64
 3   co2            2132 non-null   float64
 4   humidityratio  2132 non-null   float64
 5   target         2132 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 100.1 KB


In [6]:
cat_col = []
num_col = []
for col in list(train.columns):
    if is_categorical_dtype(train[col]):
        cat_col.append(col)
    elif is_numeric_dtype(train[col]):
        num_col.append(col)

In [7]:
num_col, len(num_col)

(['temperature', 'humidity', 'light', 'co2', 'humidityratio', 'target'], 6)

In [8]:
train

,temperature,humidity,light,co2,humidityratio,target
0,22.790000,30.8900,444.0,1372.571429,0.005307,1
1,20.890000,25.4175,0.0,573.000000,0.003879,0
2,22.100000,29.7000,0.0,1092.250000,0.004889,0
3,20.633333,24.2300,0.0,464.333333,0.003639,0
4,20.790000,24.9725,0.0,525.500000,0.003787,0
...,...,...,...,...,...,...
2127,22.660000,30.8300,433.0,1306.500000,0.005254,1
2128,20.790000,24.7900,433.0,668.000000,0.003759,1
2129,21.100000,24.8900,444.0,740.800000,0.003848,1
2130,22.200000,27.5400,538.6,1093.200000,0.004559,1


In [9]:
#train["target"] = train["target"].replace(["Kecimen", "Besni"], [0,1])

In [10]:
#ordi = OrdinalEncoder(categories=[["L", "M", ])
minmax = MinMaxScaler()
onehot = OneHotEncoder()

In [11]:
#all_col = num_col.copy()
#all_col.append("type")

In [12]:
ct = ColumnTransformer([#("ord", ordi, ["type"]),
                        ("minmax", minmax, num_col)], remainder="passthrough")

In [13]:
#new_ct = ColumnTransformer([("minmax", minmax, ["type")], remainder="passthrough")

In [14]:
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('minmax', MinMaxScaler(),
                                 ['temperature', 'humidity', 'light', 'co2',
                                  'humidityratio', 'target'])])

In [15]:
train_use = ct.transform(train)
new_features = list(ct.get_feature_names_out())
for i in range(len(new_features)):
    new_features[i] = new_features[i].split("__")[1]
train_df = pd.DataFrame(train_use, columns= new_features)
#train_df["type"] = minmax.fit_transform(np.array(train_df["type"]).reshape(-1,1))
train_df = train_df.clean_names()

In [16]:
#test["target"] = test["target"].replace(["Kecimen", "Besni"], [0,1])
test_use = ct.transform(test)
#new_features = list(ct.get_feature_names_out())
test_df = pd.DataFrame(test_use, columns=new_features)
#test_df["type"] = minmax.fit_transform(np.array(test_df["type"]).reshape(-1,1))

In [17]:
test_df = test_df.clean_names()

In [18]:
train_df.columns

Index(['temperature', 'humidity', 'light', 'co2', 'humidityratio', 'target'], dtype='object')

In [19]:
train_df

,temperature,humidity,light,co2,humidityratio,target
0,0.627119,0.937850,0.261600,0.969553,0.965656,1.0
1,0.167070,0.353961,0.000000,0.149269,0.277599,0.0
2,0.460048,0.810883,0.000000,0.681970,0.764512,0.0
3,0.104923,0.227261,0.000000,0.037787,0.161605,0.0
4,0.142857,0.306482,0.000000,0.100539,0.233320,0.0
...,...,...,...,...,...,...
2127,0.595642,0.931448,0.255119,0.901770,0.940422,1.0
2128,0.142857,0.287010,0.255119,0.246730,0.219897,1.0
2129,0.217918,0.297679,0.261600,0.321416,0.262490,1.0
2130,0.484262,0.580421,0.317337,0.682944,0.605299,1.0


In [20]:
test_df

,temperature,humidity,light,co2,humidityratio,target
0,0.748184,0.393705,0.344557,0.478995,0.606309,1.0
1,0.610775,0.353961,0.404036,0.542447,0.500649,1.0
2,0.131961,0.048013,0.000000,0.032829,0.050657,0.0
3,0.460048,0.667556,0.322286,0.804309,0.657110,1.0
4,0.329298,0.660443,0.000000,0.460528,0.578445,0.0
...,...,...,...,...,...,...
528,0.142857,0.309416,0.000000,0.093460,0.235343,0.0
529,0.167070,0.365431,0.000000,0.147833,0.285558,0.0
530,0.167070,0.309416,0.000000,0.132598,0.246695,0.0
531,0.737288,0.397439,0.104876,0.570916,0.603294,0.0


In [21]:
train_df.to_csv("../data/train_fe.csv", index=False)
test_df.to_csv("../data/test_fe.csv", index=False)